## Packages

In [1]:
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import tqdm

import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf

import matplotlib.pyplot as plt
%matplotlib inline

import numpy.linalg

import time
import math
from sympy import Matrix
from sympy.physics.quantum import TensorProduct

## MCMC Silmulation part for a single Q-bit

### Pure states

In [2]:
def facto(n):
    if n == 0 or n==1:
        return 1
    else:
        return n * facto(n-1)

def Likelyhood_multi_ratio(r_1,r_2, list_Mesure, list_instance): #r_1 would be the proposed state, and r_2 the initial one
    prod = 1
    N = len(list_instance)
    sx = np.matrix([[0 , 1] , [1 , 0]]) # introduction of Bloch matrix in order to pass from r to rho
    sy = np.matrix([[0 , -1j] , [1j , 0]])
    sz = np.matrix([[1,0] , [0,-1]])
    s = [sx, sy, sz]
    rho_1 = np.matrix([[1 , 0],[0 , 1]])/2 + r_1[0]*sx /2 + r_1[1]*sy/2 + r_1[2]*sz/2 
    rho_2 = np.matrix([[1 , 0],[0 , 1]])/2 + r_2[0]*sx /2 + r_2[1]*sy/2 + r_2[2]*sz/2 
    for i in range(0, N, 2): # mesurement are gouped by 2 for each configuration
        ng0 = list_instance[i]
        ng1 = list_instance[i+1]
        prod *= (np.trace(rho_1 @ list_Mesure[i]) / np.trace(rho_2 @ list_Mesure[i]) ) **  ng0 *  (np.trace(rho_1 @ list_Mesure[i+1]) / np.trace(rho_2 @ list_Mesure[i+1]) ) **ng1
    return prod

def MCMC_multi( list_Mesure, list_instance, N, var, r0 ):
    parcour = [ [0,0,0] for k in range(N)]
    r = r0/np.linalg.norm(r0, ord=None, axis=None, keepdims=False)
    Acceptance_count = 0 # to count in the end the acceptance rate
    for i in range(N):
        parcour[i] = r # at each iteration I store the vector r
        dr = np.random.normal(loc=0.0, scale=var, size=(1,3)) # infenitisimal shift
        r2 = r + dr[0] # I need to put dr[0] to conserve the array shape, indeed dr is a matrix(1,3)
        r2 = r2/np.linalg.norm(r2, ord=None, axis=None, keepdims=False)
        ratio = Likelyhood_multi_ratio(r2,r, list_Mesure, list_instance)
        U = np.random.uniform(low=0.0, high=1.0, size=None)
        if U < ratio:  #I shift the position according to the MH algorithm
            r = r2
            Acceptance_count += 1
    return parcour, Acceptance_count/N

In [3]:
import qutip
import mayavi
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

def proposed_state_visualisation_pure(r0, var, N):
    parcour = [ [0,0,0] for k in range(N)]
    for i in range(N):
        dr = np.random.normal(loc=0.0, scale=var, size=(1,3)) # infenitisimal shift
        r = r0 + dr[0] #r0 shifted
        r = r/np.linalg.norm(r, ord=None, axis=None, keepdims=False)
        parcour[i] = r #add the proposed state to the list
    b = qutip.Bloch3d()
    xlist = [np.float32(parcour[i][0]) for i in range(N)]
    ylist = [np.float32(parcour[i][1]) for i in range(N)]
    zlist = [np.float32(parcour[i][2]) for i in range(N)]
    b.add_points([xlist, ylist, zlist]) # proposed states on the bloch sphere
    b.point_color=['#3a7cbd']
    b.point_size=0.035
    b.add_vectors(r0)
    b.vector_color=['grey']
    b.show() #plot

In [11]:
proposed_state_visualisation_pure([0.,1.,0.] , 0.3 , 1000)


### Mixed states

In [3]:
def facto(n):
    if n == 0 or n==1:
        return 1
    else:
        return n * facto(n-1)

def Likelyhood_multi_ratio(r_1,r_2, list_Mesure, list_instance): #r_1 would be the proposed state, and r_2 the initial one
    prod = 1
    N = len(list_instance)
    sx = np.matrix([[0 , 1] , [1 , 0]]) # introduction of Bloch matrix in order to pass from r to rho
    sy = np.matrix([[0 , -1j] , [1j , 0]])
    sz = np.matrix([[1,0] , [0,-1]])
    s = [sx, sy, sz]
    rho_1 = np.matrix([[1 , 0],[0 , 1]])/2 + r_1[0]*sx /2 + r_1[1]*sy/2 + r_1[2]*sz/2 
    rho_2 = np.matrix([[1 , 0],[0 , 1]])/2 + r_2[0]*sx /2 + r_2[1]*sy/2 + r_2[2]*sz/2 
    for i in range(0, N, 2): # mesurement are gouped by 2 for each configuration
        ng0 = list_instance[i]
        ng1 = list_instance[i+1]
        prod *= (np.trace(rho_1 @ list_Mesure[i]) / np.trace(rho_2 @ list_Mesure[i]) ) **  ng0 *  (np.trace(rho_1 @ list_Mesure[i+1]) / np.trace(rho_2 @ list_Mesure[i+1]) ) **ng1
    return prod

def MCMC_multi( list_Mesure, list_instance, N, var, r0):
    parcour = [ [0,0,0] for k in range(N)]
    r = r0
    Acceptance_count = 0 # to count in the end the acceptance rate
    for i in range(N):
        parcour[i] = r # at each iteration I store the vector r
        dr = np.random.normal(loc=0.0, scale=var, size=(1,3)) # infenitisimal shift
        r2 = r + dr[0] # I need to put dr[0] to conserve the array shape, indeed dr is a matrix(1,3)
        while np.linalg.norm(r2, ord=None, axis=None, keepdims=False)>1:  #as long as we don't have a valide state, we resample a proposed state
          dr = np.random.normal(loc=0.0, scale=var, size=(1,3))
          r2 = r + dr[0]
        ratio = Likelyhood_multi_ratio(r2,r, list_Mesure, list_instance)
        U = np.random.uniform(low=0.0, high=1.0, size=None)
        if U < ratio:  #I shift the position according to the MH algorithm
            r = r2
            Acceptance_count += 1
    return parcour, Acceptance_count/N

####Proposed state plot

In [13]:
import qutip

def proposed_state_visualisation(r0, var, N):
    parcour = [ [0,0,0] for k in range(N)]
    for i in range(N):
        dr = np.random.normal(loc=0.0, scale=var, size=(1,3)) # infenitisimal shift
        r = r0 + dr[0] #r0 shifted
        r = r/( max(np.linalg.norm(r, ord=None, axis=None, keepdims=False), 1) )
        parcour[i] = r #add the proposed state to the list
    b = qutip.Bloch3d()
    xlist = [parcour[i][0] for i in range(N)]
    ylist = [parcour[i][1] for i in range(N)]
    zlist = [parcour[i][2] for i in range(N)]
    b.add_points([xlist, ylist, zlist]) # proposed states on the bloch sphere
    b.point_color=['#3a7cbd']
    b.point_size=0.035
    b.add_vectors(r0)
    b.vector_color=['grey']
    b.show() #plot


In [14]:
proposed_state_visualisation([0.9,0.,0.] , 0.1 , 500)

### Example and plot

In [4]:
# x, y, z
ex1 = [{'0': 75, '1': 0}, {'0': 29, '1': 46}, {'1': 35, '0': 40}]
# outcomes = [0] * ex1[0]['0'] + [1] * ex1[0]['1'] + [2] * ex1[1]['0'] + [3] * ex1[1]['1'] + [4] * ex1[2]['0'] + [5] * ex1[2]['1']

S0 = np.array([[1],[0]])
S1 = np.array([[0],[1]])

Rho0 = S0 @ S0.T
Rho1 = S1 @ S1.T


E0 = np.matrix([[1 , 0],[0 , 0]])
E1 = np.matrix([[0 , 0],[0 , 1]])

H = 1/np.sqrt(2) * np.matrix([[1 , 1],[1 , -1]])
H0 = H @ S0 @ S0.T @ H.H
H1 = H @ S1 @ S1.T @ H.H


U = 1/np.sqrt(2) * np.matrix([[1 , -1j],[1 , 1j]])
U0 = np.linalg.inv(U) @ S0 @ S0.T @ np.linalg.inv(U).H
U1 = np.linalg.inv(U) @ S1 @ S1.T @ np.linalg.inv(U).H

In [ ]:
#!pip install qiskit
from qiskit import *
from qiskit.tools.monitor import job_monitor
from numpy import pi
from qiskit import Aer
from qiskit.circuit.random import random_circuit
import qiskit.quantum_info as qi

number_of_qubits=1

class experiments():
    def __init__(self,n_of_shots,n_of_qubits,real=0,random=0):
        self.n_of_shots=n_of_shots
        self.n_of_qubits=n_of_qubits
        self.random=random
        self.operators=[]
        self.real_state=0
        
        self.shots=n_of_shots
        if real==0:
            self.simulated=[0]*(2**n_of_qubits+1)
            for i in range(2**n_of_qubits+1):
              self.simulate(i)
        if real==1:
            self.res=[0]*(2**n_of_qubits+1)
            for i in range(2**n_of_qubits+1):
              self.rl(i)
        
    def simulate(self,op_number):
        qr=QuantumRegister(self.n_of_qubits)
        cr=ClassicalRegister(self.n_of_qubits)
        circuit=QuantumCircuit(qr,cr)

        self.real_state=np.matrix([[1 if i==j==0 else 0 for i in range(2**self.n_of_qubits)] for j in range(2**self.n_of_qubits)])
        for i in range(self.n_of_qubits):
            circuit.h(qr[i])
            circuit.barrier(qr[i])
        
        self.real_state=np.matmul(np.matrix(qi.Operator(circuit)),np.matmul(self.real_state,np.matrix(qi.Operator(circuit)).H))
        if self.random==0:
            qr1=QuantumRegister(self.n_of_qubits)
            cr1=ClassicalRegister(self.n_of_qubits)
            circuit1=QuantumCircuit(qr1,cr1)
                
            if op_number<2**self.n_of_qubits:
                for i in range(self.n_of_qubits):
                  if(vec[op_number][i]=="0"):
                      circuit.h(qr[i])
                      circuit1.h(qr1[i])
                  else:
                      circuit1.u(pi / 2, 0, pi / 2, qr1[i])
                      circuit.u(pi / 2, 0, pi / 2, qr[i])
                self.operators.append(qi.Operator(circuit1))
                
            else:
                self.operators.append(qi.Operator(circuit1))

        
        else:
            circuit1=random_circuit(self.n_of_qubits, 3,3)
            self.operators.append(qi.Operator(circuit1))
            circuit.compose(circuit1,inplace=True)
            
        circuit.measure(qr,cr)

        simulator=Aer.get_backend("qasm_simulator")
        result=execute(circuit,backend=simulator,shots=self.shots).result()
        self.simulated[op_number]=result.get_counts(circuit)
        
    def rl(self,op_number):
        qr=QuantumRegister(self.n_of_qubits)
        cr=ClassicalRegister(self.n_of_qubits)
        circuit=QuantumCircuit(qr,cr)
        
        for i in range(self.n_of_qubits):
            circuit.h(qr[i])
            circuit.barrier(qr[i])
        
        if self.random==0:
            qr1=QuantumRegister(self.n_of_qubits)
            cr1=ClassicalRegister(self.n_of_qubits)
            circuit1=QuantumCircuit(qr1,cr1)
                
            if op_number<2**self.n_of_qubits:
                for i in range(self.n_of_qubits):
                  if(vec[op_number][i]=="0"):
                      circuit.h(qr[i])
                      circuit1.h(qr1[i])
                  else:
                      circuit1.u(pi / 2, 0, pi / 2, qr1[i])
                      circuit.u(pi / 2, 0, pi / 2, qr[i])
                self.operators.append(qi.Operator(circuit1))
                
            else:
                self.operators.append(qi.Operator(circuit1))

        
        else:
            circuit1=random_circuit(self.n_of_qubits, 3,3)
            self.operators.append(qi.Operator(circuit1))
            circuit.compose(circuit1,inplace=True)
            
        circuit.measure(qr,cr)
        
        IBMQ.load_account()
        provider=IBMQ.get_provider("ibm-q")
        qcomp=provider.get_backend("ibmq_belem")
        job=execute(circuit,backend=qcomp,shots=self.shots)
        job_monitor(job)
        result=job.result()
        self.res[op_number]=result.get_counts(circuit)
        
ex1=experiments(300, number_of_qubits,0,0)
for i in range(2**number_of_qubits+1):
  print(ex1.simulated[i])
  print("------------")
  print(ex1.operators[i])

In [ ]:
#multi state estimation
list,rate = MCMC_multi([H0, H1, U0, U1, E0, E1],
                      [75, 0, 29, 46, 35,40], 
                      10000, 0.05, 
                      [0,0,0])

In [10]:
#this for the plot
list,rate = MCMC_multi([H0, H1, U0, U1, E0, E1],
                      [284, 16, 149, 151, 151, 149], 
                      10000, 0.05, 
                      [0,0,0])

NameError: name 'H0' is not defined

In [ ]:
#state 0 multi
list,rate = MCMC_multi([H0, H1, U0, U1, E0, E1],
                      [0,0,0,0,10,0], 
                      5000, 0.3, 
                      [0.5,0.5,0.5])

In [ ]:
#state 1 mutli
list,rate = MCMC_multi([H0, H1, U0, U1, E0, E1],
                      [0,0,0,0,0,10], 
                      5000, 0.3, 
                      [0.5,0.5,0.5])

In [ ]:
# test mutli
list,rate = MCMC_multi([H0, H1, U0, U1, E0, E1],
                      [30,30,30,30,30,30], 
                      5000, 0.1, 
                      [0.5,0.5,0.5])

In [ ]:
import qutip
N = len(list)
Burn = 5000
# N = len(states)
# Burn = int(N*0.1)

# z = [states[i][2] for i in range(Burn,N)]
# x = [states[i][0] for i in range(Burn,N)]
# y = [states[i][1] for i in range(Burn,N)]

b = qutip.Bloch3d()
xlist = [list[i][0] for i in range(Burn,N)]
ylist = [list[i][1] for i in range(Burn,N)]
zlist = [list[i][2] for i in range(Burn,N)]
b.add_points([xlist, ylist, zlist]) # proposed states on the bloch sphere
b.point_color=['#3a7cbd']
b.point_size=0.035
b.add_vectors([0.9,0.,0.])
b.vector_color=['grey']
b.show() #plot


### Quiskit simulator

In [ ]:
number_of_qubits=1

def to_string(lis):
  strn=""
  l=[0 for i in range(len(lis))]
  for i in range(len(lis)):
    if(lis[i]-1>=0):
      l[lis[i]-1]=1
  for i in reversed(range(len(l))):
    strn+=str(l[i])
  return strn

possible_values=dict()

def values(lis,n):
  possible_values[to_string(lis)]=0
  if lis[0]==n:
    return
  i=n-1
  while(i>=0 and lis[i]+1>n):
    lis[i]=0
    i=i-1
  lis[i]=lis[i]+1
  values(lis,n)

values([0 for i in range((number_of_qubits))],(number_of_qubits))

vec = [elem for elem in possible_values]

def frobenius_coordinates(state,pauli):
  return np.real(np.trace(np.matmul(state.H,pauli)))

def find_radius(cor):
  sum=0
  for i in range(len(cor)):
    sum=sum+cor[i]**2
  return np.sqrt(sum) 

#!pip install qiskit
from qiskit import *
from qiskit.tools.monitor import job_monitor
from numpy import pi
from qiskit import Aer
from qiskit.circuit.random import random_circuit
import qiskit.quantum_info as qi



class experiments():
    def __init__(self,n_of_shots,n_of_qubits,real=0,random=0):
        self.n_of_shots=n_of_shots
        self.n_of_qubits=n_of_qubits
        self.random=random
        self.operators=[]
        self.real_state=0
        
        self.shots=n_of_shots
        if real==0:
            self.simulated=[0]*(2**n_of_qubits+1)
            for i in range(2**n_of_qubits+1):
              self.simulate(i)
        if real==1:
            self.res=[0]*(2**n_of_qubits+1)
            for i in range(2**n_of_qubits+1):
              self.rl(i)
        
    def simulate(self,op_number):
        qr=QuantumRegister(self.n_of_qubits)
        cr=ClassicalRegister(self.n_of_qubits)
        circuit=QuantumCircuit(qr,cr)

        self.real_state=np.matrix([[1 if i==j==0 else 0 for i in range(2**self.n_of_qubits)] for j in range(2**self.n_of_qubits)])
        for i in range(self.n_of_qubits):
            circuit.h(qr[i])
            circuit.barrier(qr[i])
        
        self.real_state=np.matmul(np.matrix(qi.Operator(circuit)),np.matmul(self.real_state,np.matrix(qi.Operator(circuit)).H))
        if self.random==0:
            qr1=QuantumRegister(self.n_of_qubits)
            cr1=ClassicalRegister(self.n_of_qubits)
            circuit1=QuantumCircuit(qr1,cr1)
                
            if op_number<2**self.n_of_qubits:
                for i in range(self.n_of_qubits):
                  if(vec[op_number][i]=="0"):
                      circuit.h(qr[i])
                      circuit1.h(qr1[i])
                  else:
                      circuit1.u(pi / 2, 0, pi / 2, qr1[i])
                      circuit.u(pi / 2, 0, pi / 2, qr[i])
                self.operators.append(qi.Operator(circuit1))
                
            else:
                self.operators.append(qi.Operator(circuit1))

        
        else:
            circuit1=random_circuit(self.n_of_qubits, 3,3)
            self.operators.append(qi.Operator(circuit1))
            circuit.compose(circuit1,inplace=True)
            
        circuit.measure(qr,cr)

        simulator=Aer.get_backend("qasm_simulator")
        result=execute(circuit,backend=simulator,shots=self.shots).result()
        self.simulated[op_number]=result.get_counts(circuit)
        
    def rl(self,op_number):
        qr=QuantumRegister(self.n_of_qubits)
        cr=ClassicalRegister(self.n_of_qubits)
        circuit=QuantumCircuit(qr,cr)
        
        for i in range(self.n_of_qubits):
            circuit.h(qr[i])
            circuit.barrier(qr[i])
        
        if self.random==0:
            qr1=QuantumRegister(self.n_of_qubits)
            cr1=ClassicalRegister(self.n_of_qubits)
            circuit1=QuantumCircuit(qr1,cr1)
                
            if op_number<2**self.n_of_qubits:
                for i in range(self.n_of_qubits):
                  if(vec[op_number][i]=="0"):
                      circuit.h(qr[i])
                      circuit1.h(qr1[i])
                  else:
                      circuit1.u(pi / 2, 0, pi / 2, qr1[i])
                      circuit.u(pi / 2, 0, pi / 2, qr[i])
                self.operators.append(qi.Operator(circuit1))
                
            else:
                self.operators.append(qi.Operator(circuit1))

        
        else:
            circuit1=random_circuit(self.n_of_qubits, 3,3)
            self.operators.append(qi.Operator(circuit1))
            circuit.compose(circuit1,inplace=True)
            
        circuit.measure(qr,cr)
        
        IBMQ.load_account()
        provider=IBMQ.get_provider("ibm-q")
        qcomp=provider.get_backend("ibmq_belem")
        job=execute(circuit,backend=qcomp,shots=self.shots)
        job_monitor(job)
        result=job.result()
        self.res[op_number]=result.get_counts(circuit)
        
ex1=experiments(300, number_of_qubits,0,0)
for i in range(2**number_of_qubits+1):
  print(ex1.simulated[i])
  print("------------")
  print(ex1.operators[i])
        
        
        
        
        
        

In [ ]:
dic = ex1.simulated

In [ ]:
dic

### correlogram

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

def plot_autorelation(data):
  N = len(data)
  dictionary = {
    'X': [data[i][0] for i in range(N)], 
    'Y': [data[i][1] for i in range(N)], 
    'Z': [data[i][2] for i in range(N)]
    }
  df = pd.DataFrame(dictionary)
  sm.graphics.tsa.plot_acf(df['X'].values.squeeze(), lags=50)
  plt.show()

In [ ]:
rate

In [ ]:
path = list
plot_autorelation(path)

### Fidelity measure

In [ ]:
import numpy as np
from scipy.linalg import sqrtm

def fidelity(r_true, list_point):
  fid = 0 #the final return
  N= len(list_point)
  sx = np.matrix([[0 , 1] , [1 , 0]]) 
  sy = np.matrix([[0 , -1j] , [1j , 0]])
  sz = np.matrix([[1,0] , [0,-1]]) # matrixes in order to convert r vector to rho matrix
  s = [sx, sy, sz]
  rho_true = np.matrix([[1 , 0],[0 , 1]])/2 + r_true[0]*sx /2 + r_true[1]*sy/2 + r_true[2]*sz/2 
  for i in range(N):
    r = list_point[i]
    rho = np.matrix([[1 , 0],[0 , 1]])/2 + r[0]*sx /2 + r[1]*sy/2 + r[2]*sz/2 #convert the r vector to the rho matrix
    M = rho @ rho_true
    fid += np.trace(sqrtm(M)).real**2 # fidelity sum
  return fid/N #return weighted fidelity sum

In [ ]:
fidelity([1,0,0], list[1000:])

In [ ]:
import numpy as np
from scipy.linalg import sqrtm
a = np.array([[1.0, 3.0], [1.0, 4.0]])
r = sqrtm(a)
r 

##MCMC simulation for 2 Q-bits

Matrix initilization

In [5]:
H=np.matrix([[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]])
U=np.matrix([[1/np.sqrt(2),-1j/np.sqrt(2)],[1/np.sqrt(2),1j/np.sqrt(2)]])
I=np.matrix(np.eye(2))
Id = np.matrix([[1 , 0],[0 , 1]])
sx = np.matrix([[0 , 1] , [1 , 0]])
sy = np.matrix([[0 , -1j] , [1j , 0]])
sz = np.matrix([[1,0] , [0,-1]])
operators=[np.matrix(TensorProduct(H,H)),np.matrix(TensorProduct(U,H)),np.matrix(TensorProduct(H,U)),np.matrix(TensorProduct(U,U)),np.matrix(TensorProduct(I,I))]
pauli_1Q=[Id,sx,sy,sz]
ang_coor=[0]*15
pauli_2Q=[]
for i in range(4):
  for j in range(4):
    pauli_2Q.append(np.matrix(TensorProduct(pauli_1Q[i],pauli_1Q[j])))

Generate sample from the real state (distribution of the real state)

In [6]:
def generate():
  for i in range(13):
    ang_coor[i]=math.pi*np.random.beta(200,200)
  ang_coor[13]=2*math.pi*np.random.beta(400,400)

  rho=np.random.beta(90,10)
  coor=[rho]*15
  for i in range(14):
    for j in range(i+1):
      if(i==j):
        coor[i]=coor[i]*math.cos(ang_coor[j])
      else:
        coor[i]=coor[i]*math.sin(ang_coor[j])

  for i in range(14):
    coor[14]=coor[14]*math.sin(ang_coor[i])

  state=pauli_2Q[0]
  for i in range(15):
    state=state+coor[i]*pauli_2Q[i+1]
  state=state/4
  return state

def measure(state):
  prob=[0]*4
  for i in range(3):
    prob[i]=np.real(state[i,i])
  prob[3]=1-prob[0]-prob[1]-prob[2]
  return  np.random.multinomial(1,prob)

pos=dict()
pos[0]="00"
pos[1]="01"
pos[2]="10"
pos[3]="11"

exp=dict()
exp["00"]=0
exp["01"]=0
exp["10"]=0
exp["11"]=0

def experiment():
  state=generate()
  results=[]
  for i in range(5):
    T_state=np.matmul(operators[i],np.matmul(state,operators[i].H))
    x=measure(T_state)
    results.append(x)
  return results

class random_experiments():
  def __init__(self,n):
    self.n=n
    self.simulated=[exp.copy() for i in range(5)]

    for j in range(self.n):
      result=experiment()
      for i in range(5):
          self.simulated[i][pos[np.where(result[i]>0)[0][0]]]=self.simulated[i][pos[np.where(result[i]>0)[0][0]]]+1

In [7]:
possible_values=dict()
number_of_qubits=2
def values(lis,n):
  possible_values[to_string(lis)]=0
  if lis[0]==n:
    return
  i=n-1
  while(i>=0 and lis[i]+1>n):
    lis[i]=0
    i=i-1
  lis[i]=lis[i]+1
  values(lis,n)

def to_string(lis):
  strn=""
  l=[0 for i in range(len(lis))]
  for i in range(len(lis)):
    if(lis[i]-1>=0):
      l[lis[i]-1]=1
  for i in reversed(range(len(l))):
    strn+=str(l[i])
  return strn

number_of_qubits =2
values([0 for i in range((number_of_qubits))],(number_of_qubits))

mesur_list = [np.zeros((4,4))] * 20
E_list = [np.zeros((4,4)),np.zeros((4,4)),np.zeros((4,4)),np.zeros((4,4)) ]
for i in range(4):
  E_list[i].itemset((i,i),1)

for j in range(5):
  for i in range(4):
    mesur_list[j * 4 + i] = np.linalg.inv(operators[j]) @ E_list[i] @ np.linalg.inv(operators[j]).H

mesur_dic = [{} for k in range(5)]
possible_outcome = [ elem for elem in possible_values]
for j in range(5):
  for i,val in enumerate(possible_outcome):
    mesur_dic[j][val] = mesur_list[j * 4 + i]

In [8]:
ex1=random_experiments(1000)

In [9]:
output_dic = ex1.simulated

In [10]:
def facto(n):
    if n == 0 or n==1:
        return 1
    else:
        return n * facto(n-1)

def Likelyhood_multi_ratio(r_1,r_2, list_Mesure, list_instance): #r_1 would be the proposed state, and r_2 the initial one
    prod = 1
    N = len(list_instance)
    sx = np.matrix([[0 , 1] , [1 , 0]]) # introduction of Bloch matrix in order to pass from r to rho
    sy = np.matrix([[0 , -1j] , [1j , 0]])
    sz = np.matrix([[1,0] , [0,-1]])
    s = [sx, sy, sz]
    rho_1 = np.matrix([[1 , 0],[0 , 1]])/2 + r_1[0]*sx /2 + r_1[1]*sy/2 + r_1[2]*sz/2 
    rho_2 = np.matrix([[1 , 0],[0 , 1]])/2 + r_2[0]*sx /2 + r_2[1]*sy/2 + r_2[2]*sz/2 
    for i in range(0, N, 2): # mesurement are gouped by 2 for each configuration
        ng0 = list_instance[i]
        ng1 = list_instance[i+1]
        prod *= (np.trace(rho_1 @ list_Mesure[i]) / np.trace(rho_2 @ list_Mesure[i]) ) **  ng0 *  (np.trace(rho_1 @ list_Mesure[i+1]) / np.trace(rho_2 @ list_Mesure[i+1]) ) **ng1
    return prod

def Likelyhood_ratio_2Q(r_1, r_2, dic_mesure, dic_output, pauli_2Q):
    prod = 1
    rho_1 = pauli_2Q[0]/4 #the identity matrix
    rho_2 = pauli_2Q[0]/4 #the identity matrix
    for j in range(15):
      rho_1 = rho_1 + pauli_2Q[j+1] * r_1[j] / 4 # add the linear combinaision in order to creat rho
      rho_2 = rho_2 + pauli_2Q[j+1] * r_2[j] / 4
    for a in range(5): # for each configuration alpha
      for gamma in dic_output[a]: # uptdate the likelyhood by iterating throw all the different ouput
        prod *= ( np.trace(rho_1 @ dic_mesure[a][gamma]) / np.trace(rho_2 @ dic_mesure[a][gamma]) )  ** dic_output[a][gamma]
    return prod

def MCMC_2Q( dic_mesure, dic_output, N, var, r0, pauli_2Q):
    parcour = [ [0]*15 for k in range(N)]
    r = r0
    Acceptance_count = 0 # to count in the end the acceptance rate
    for i in range(N):
        parcour[i] = r # at each iteration I store the vector r
        dr = np.random.normal(loc=0.0, scale=var, size=(1,15)) # infenitisimal shift
        r2 = r + dr[0] # I need to put dr[0] to conserve the array shape, indeed dr is a matrix(1,3)
        while np.linalg.norm(r2, ord=None, axis=None, keepdims=False)>1:  #as long as we don't have a valide state, we resample a proposed state
          dr = np.random.normal(loc=0.0, scale=var, size=(1,15))
          r2 = r + dr[0]
        ratio = Likelyhood_ratio_2Q(r2, r, dic_mesure, dic_output, pauli_2Q)
        U = np.random.uniform(low=0.0, high=1.0, size=None)
        if U < ratio:  #I shift the position according to the MH algorithm
            r = r2
            Acceptance_count += 1
    return parcour, Acceptance_count/N

In [11]:
parcour,rate = MCMC_2Q(mesur_dic, output_dic,40000,0.01,[0]*15,pauli_2Q)

#### plot framework

In [22]:
len(parcour)

40000

In [23]:
def partial_trace(state):
    S=np.matrix([[0+1j*0,0+1j*0],[0+1j*0,0+1j*0]])
    S[0,0]=state[0,0]+state[1,1]
    S[0,1]=state[0,2]+state[1,3]
    S[1,0]=state[2,0]+state[3,1]
    S[1,1]=state[2,2]+state[3,3]
    return S

In [12]:
def to_state(coor):
    state=pauli_2Q[0]
    for i in range(len(coor)):
        state=state+coor[i]*pauli_2Q[i+1]
    state=state/4
    return state

In [13]:
states=[to_state(i) for i in parcour]

In [26]:
p_trace = [partial_trace(i) for i in states]

In [14]:
def measure(ρ):
    Id = np.matrix([[1 , 0],[0 , 1]])
    sx = np.matrix([[0 , 1] , [1 , 0]])
    sy = np.matrix([[0 , -1j] , [1j , 0]])
    sz = np.matrix([[1,0] , [0,-1]])    
    x = np.trace(np.dot(ρ, sx)).real
    y = np.trace(np.dot(ρ, sy)).real
    z = np.trace(np.dot(ρ, sz)).real
    return [x, y, z]

In [15]:
def partial_trace_1(state):
    S=np.matrix([[0+1j*0,0+1j*0],[0+1j*0,0+1j*0]])
    S[0,0]=state[0,0]+state[2,2]
    S[0,1]=state[0,1]+state[2,3]
    S[1,0]=state[1,0]+state[3,2]
    S[1,1]=state[1,1]+state[3,3]
    return S

In [16]:
p_trace = [partial_trace_1(i) for i in states]

In [ ]:
import qutip
#plot with color for weight visualisation
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
 
fig = plt.figure()

# max = np.amax(weigths)

b = qutip.Bloch3d()
x, y, z = [], [], []
# syntax for 3-D projection
for i in range(35000,40000):
  #c = ((1-weigths[i]/max),(1-weigths[i]/max), (1-weigths[i]/max))
    c = (0,0,0)
    p = measure(p_trace[i])
    z.append(p[2])
    x.append(p[0])
    y.append(p[1])
  #ax.scatter(x, y, z, c = weights[i]*255*10)


b.add_points([x,y,z],alpha=1)
b.point_color=['#3a7cbd']
b.point_size=0.035

b.show()

#### fidelity

In [ ]:
import numpy as np
from scipy.linalg import sqrtm


def fidelity_2Q(list_point,pauli_2Q):
  fid = 0 #the final return
  N= len(list_point)
  for i in range(N):
    r = list_point[i]
    rho = pauli_2Q[0]/4 #the identity matrix
    for j in range(15):
      rho = rho + pauli_2Q[j+1] * r[j] / 4
    M = rho @ generate()
    fid += np.trace(sqrtm(M)).real**2 # fidelity sum
  return fid/N #return weighted fidelity sum

In [ ]:
fidelity_2Q(parcour[10000:], pauli_2Q)